# SEIRX model runs for test technology and screening frequency investigation in schools

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from os.path import join
import os
import shutil
import pickle
import json

import sys
sys.path.insert(0,'../school')
sys.path.insert(0,'../nursing_home')
from model_school import SEIRX_school
import analysis_functions as af

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

**Note:** The number of simulation runs per scenario is set via the ```runs``` variable below. Running 10k simulations per scenario takes approximately 12 hours on a single ~4GHz core. Therefore, if you just want to test the simulation, I advise to set ```runs``` to for example 100. This will result in noisier statistics, but the trends will remain the same. 

In [2]:
# number of runs per scenario
runs = 2
# this is where simulation results will be stored
res_path = '../data/school' 
# file name of the results, will be appended by the number of runs
school_name = 'test_volksschule'
school_type = 'volksschule'
classes = 10
students = 20
floors = 2
mask = False
half_classes = False

# agent contact network
G = nx.readwrite.gpickle.read_gpickle(join(res_path,\
                                        '{}.gpickle'.format(school_name)))
# teacher schedule
schedule = pd.read_csv(join(res_path,'{}_schedule.csv'.format(school_name)))
schedule.index = schedule['teacher']
schedule = schedule.drop(columns=['teacher'])
# list of nodes (agents) in the contact network and their attributes
node_list = pd.read_csv(join(res_path,'{}_nodelist.csv'.format(school_name)))

# maximum number of steps in a single run. A run automatically stops
# if the outbreak is contained, i.e. there are no more infected or
# exposed agents
N_steps = 100 
# student and teacher streening intervals (in days)
s_screen_range = [3]
t_screen_range = [3]
testing = 'preventive'
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen']
# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['student', 'teacher']
# agent types used in the simulation with their respective parameters
agent_types = {
        'student':{
            'screening_interval': None,
            'index_probability': 0,
            'transmission_risk': 0.1,
            'reception_risk': 1,
            'symptom_probability': 0.6,
            'mask':False},
    
        'teacher':{
            'screening_interval': 7,
            'index_probability': 0,
            'transmission_risk': 0.1,
            'reception_risk': 1,
            'symptom_probability': 0.6,
            'mask':True},
    
        'family_member':{
            'screening_interval': None,
            'index_probability': 0,
            'transmission_risk': 0.1,
            'reception_risk': 1,
            'symptom_probability': 0.6,
            'mask':False}
}

# progress bar
f = IntProgress(min=0, max=runs * len(s_screen_range) \
                * len(t_screen_range) * len(test_types) * len(index_cases)) 
display(f)
c=0 # counter for progress bar

scan_results = pd.DataFrame()

for ttype in test_types:
    for index_case in index_cases:
        for s_screen_interval in s_screen_range:
            agent_types['student']['screening_interval'] = s_screen_interval
            for t_screen_interval in t_screen_range:
                agent_types['teacher']['screening_interval'] = t_screen_interval
                
                # results of one ensemble, i.e. results of all runs
                # with the same parameters
                ensemble_results = pd.DataFrame()
                try:
                    shutil.rmtree(join(res_path, 'tmp'))
                except FileNotFoundError:
                    pass
                os.mkdir(join(res_path, 'tmp'))
                class_path = 'schooltype-{}_classes-{}_students-{}_floors-{}'\
                    .format(school_type, classes, students, floors)
                
                try:
                    os.mkdir(join(res_path, class_path))
                except FileExistsError:
                    pass
                
                for r in range(runs):
                    f.value = c # update the progress bar
                    c += 1
                    print(c)
                    
                    # instantiate model with current scenario settings
                    model = SEIRX_school(G, testing=testing,
                      diagnostic_test_type = 'two_day_PCR',
                      preventive_screening_test_type = ttype,
                      exposure_duration = [5, 1.9],
                      time_until_symptoms = [6.4, 0.8],
                      infection_duration = [10.91, 3.95],
                      index_case = index_case,
                      agent_types = agent_types)
                    
                    # run the model, end run if the outbreak is over
                    for i in range(N_steps):
                        model.step()
                        if len([a for a in model.schedule.agents if \
                            (a.exposed == True or a.infectious == True)]) == 0:
                            break
    
                    # collect the statistics of the single run
                    row = af.get_ensemble_observables_school(model, r)
                    # add run results to the ensemble results
                    ensemble_results = ensemble_results.append(row,
                        ignore_index=True)
                    
                    N_infected = row['infected_agents']
                    
                    with open(join(join(res_path, 'tmp'),\
                                   'run_{}_N_{}.p'.format(r, N_infected)), 'wb') as f:
                        pickle.dump(model, f)
                    
               # add ensemble statistics to the overall results
                row = {'test_type':ttype,
                       'index_case':index_case,
                       'student_screen_interval':s_screen_interval,
                       'teacher_screen_interval':t_screen_interval}

                row.update(af.get_statistics(ensemble_results, 'R0'))
                row.update(af.get_statistics(ensemble_results, 'infected_students'))
                row.update(af.get_statistics(ensemble_results, 'infected_teachers'))
                row.update(af.get_statistics(ensemble_results, 'infected_family_members'))
                row.update(af.get_statistics(ensemble_results, 'infected_agents'))
                row.update(af.get_statistics(ensemble_results, 'N_diagnostic_tests'))
                row.update(af.get_statistics(ensemble_results, 'N_preventive_tests'))
                row.update(af.get_statistics(ensemble_results, 'transmissions'))
                row.update(af.get_statistics(ensemble_results, 'infected_without_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_student_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'teacher_student_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_teacher_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'teacher_teacher_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_family_member_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'family_member_family_member_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_student'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_teacher'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_family_member'))
                row.update(af.get_statistics(ensemble_results, 'pending_test_infections'))
                row.update(af.get_statistics(ensemble_results, 'undetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'predetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'duration'))

                scan_results = scan_results.append(row, ignore_index=True)
                
                rep_model = af.get_representative_run(row['infected_agents_median'],\
                                                 join(res_path, 'tmp'))
                tm_events = af.get_transmission_chain(rep_model, schedule)
                
                af.dump_JSON(join(res_path, class_path),
                          school_type, classes, students, floors,
                          ttype, index_case, s_screen_interval,
                          t_screen_interval, mask, half_classes,
                          G, node_list, schedule, row, tm_events)
                
                shutil.rmtree(join(res_path, 'tmp'))
            
# save results to disk
scan_results.to_csv(join(res_path,'{}_N{}.csv'.format(school_name, runs)), index=False)

IntProgress(value=0, max=4)

1
2


../school/agent_student.py:79: RuntimeWarning: invalid value encountered in double_scalars
  modifier = 1 - max(0, self.days_since_exposure - self.exposure_duration - 1) / \


3


/home/jana/anaconda3/envs/covid/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jana/anaconda3/envs/covid/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


4
